# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_api

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_pd = pd.read_csv("City1.csv")
cities_pd.dropna(inplace=True)

cities_pd.head()
print(len(cities_pd))

563


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_api)
locations = cities_pd[["Lat", "Long"]]

center = (0,0)

# Fill NaN values and convert to float
humidity = cities_pd["Humidity"].astype(float)

fig = gmaps.figure(center=center, zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                 max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
cities_pd
hotel_df = cities_pd[(cities_pd['Max Temperature']<80)&(cities_pd['Max Temperature']>70)&(cities_pd['Wind Speed']<5)&(cities_pd['Cloudiness']<1)].copy()
hotel_df

,Lat,Long,City,Humidity,Temperature,Cloudiness,Wind Speed,Max Temperature,Country,Date
104,16.3333,-15.0000,saint-louis,46.0,75.20,0.0,2.30,75.20,SN,1.615973e+09
162,14.8961,-24.4956,sao filipe,72.0,71.22,0.0,4.88,71.22,CV,1.615973e+09
179,-33.0117,17.9442,saldanha,68.0,78.80,0.0,2.30,78.80,ZA,1.615973e+09
214,29.9697,40.2064,sakakah,18.0,77.00,0.0,4.61,77.00,SA,1.615973e+09
528,-0.1150,-51.2894,mazagao,100.0,75.20,0.0,4.61,75.20,BR,1.615973e+09
538,-17.5333,-149.3333,tiarei,73.0,78.40,0.0,4.61,78.80,PF,1.615973e+09
578,-22.2833,166.5833,mont-dore,73.0,77.00,0.0,4.61,77.00,NC,1.615973e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df["Hotel Name"] = ""
hotel_df

,Lat,Long,City,Humidity,Temperature,Cloudiness,Wind Speed,Max Temperature,Country,Date,Hotel Name
104,16.3333,-15.0000,saint-louis,46.0,75.20,0.0,2.30,75.20,SN,1.615973e+09,
162,14.8961,-24.4956,sao filipe,72.0,71.22,0.0,4.88,71.22,CV,1.615973e+09,
179,-33.0117,17.9442,saldanha,68.0,78.80,0.0,2.30,78.80,ZA,1.615973e+09,
214,29.9697,40.2064,sakakah,18.0,77.00,0.0,4.61,77.00,SA,1.615973e+09,
528,-0.1150,-51.2894,mazagao,100.0,75.20,0.0,4.61,75.20,BR,1.615973e+09,
538,-17.5333,-149.3333,tiarei,73.0,78.40,0.0,4.61,78.80,PF,1.615973e+09,
578,-22.2833,166.5833,mont-dore,73.0,77.00,0.0,4.61,77.00,NC,1.615973e+09,


In [6]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_api
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index,"Hotel Name"] = name_address["results"][0]["name"]
       
        #print(f'Processing record for City: {name_address["name"]}')
    except:
        print("Missing field/result... skipping.")


    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.


In [7]:
hotel_df

,Lat,Long,City,Humidity,Temperature,Cloudiness,Wind Speed,Max Temperature,Country,Date,Hotel Name
104,16.3333,-15.0000,saint-louis,46.0,75.20,0.0,2.30,75.20,SN,1.615973e+09,La cour du fleuve
162,14.8961,-24.4956,sao filipe,72.0,71.22,0.0,4.88,71.22,CV,1.615973e+09,Hotel Miramar Fogo Brava
179,-33.0117,17.9442,saldanha,68.0,78.80,0.0,2.30,78.80,ZA,1.615973e+09,The Shark Bay Hotel & Spa
214,29.9697,40.2064,sakakah,18.0,77.00,0.0,4.61,77.00,SA,1.615973e+09,Admato Hotel
528,-0.1150,-51.2894,mazagao,100.0,75.20,0.0,4.61,75.20,BR,1.615973e+09,ROYAL hotel e gastronomia
538,-17.5333,-149.3333,tiarei,73.0,78.40,0.0,4.61,78.80,PF,1.615973e+09,Hotel Manava Beach Resort & Spa Moorea
578,-22.2833,166.5833,mont-dore,73.0,77.00,0.0,4.61,77.00,NC,1.615973e+09,Escapade Îlot Maître


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]


In [9]:
# Add marker layer ontop of heat map
pins = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(pins)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))